NOMBRE: Priscilla González Sandoval


CARNE: 20689


FECHA: 22.09.24

Responda las siguientes preguntas:
1. ¿Cuáles son los dos procesos principales de un transformer y qué función cumplen?
  
    *   A. Encoding:
El input que se ingresa que en este caso puede ser cualquoer tipo de datp es transformado en una representación interna o codificación. La principal función del encoding es generar una representación contextualizada de cada elemento del input, teniendo en cuenta las relaciones entre las palabras o elementos dentro de la secuencia, utilizando mecanismos como la self-attention.

    *   B. Decoding:
Este toma la representación generada por el encoding y lo que hace es que la transforma en el output deseado. Mientras este proceso se realiza, el decoding también utiliza self-attention para generar cada token del output, prestando atención tanto al input como a la secuencia generada previamente.
  
2. Asigne cada uno de las siguientes subtareas a su proceso correspondiente(A o B): Multi-Head Self-Attention Mechanism,Position-wise Feed-Forward Networks,Masked Multi-Head Self-Attention Mechanism, Encoder-Decoder Multi-Head Attention,Position-wise Feed-Forward Networks

    *   A. Codificación (Encoding):

        *     Multi-Head Self-Attention Mechanism
        *     Position-wise Feed-Forward Networks
        
    *   B. Decodificación (Decoding):

        *     Masked Multi-Head Self-Attention Mechanism
        *     Encoder-Decoder Multi-Head Attention
        *     Position-wise Feed-Forward Networks

Después de asignar, explique la función de cada una de las partes. Luego, proceda a programar un transformer con cada una de esas partes utilizando solo tensorflow o Pytorch.

- Multi-Head Self-Attention Mechanism:

    - Permite al modelo centrarse en diferentes posiciones de la secuencia del input simultáneamente. Cada "head" de atención obtiene una representación diferente de la información en cada token, lo que ayuda al modelo a entender mejor el contexto global. En la codificación, esto sirve para el análisis de las relaciones dentro de la secuencia de entrada.

- Position-wise Feed-Forward Networks:

    - Estas redes aplican una capa de feed-forward completamente conectada a cada token de manera independiente, tras las capas de atención. Actúan como transformaciones no lineales para mejorar la capacidad de representación de las capas de atención.

- Masked Multi-Head Self-Attention Mechanism (Decodificación):

    - Similar al Multi-Head Self-Attention Mechanism, solo que en este caso se utiliza una máscara para evitar que el modelo "vea" las posiciones futuras de la secuencia. De esta forma, el modelo genera un token a la vez.

- Encoder-Decoder Multi-Head Attention (Decodificación):

    - Esta capa permite que el decodificador preste atención a la secuencia generada por el codificador. Ayuda a que el modelo se enfoque en las partes más relevantes de la secuencia de entrada mientras genera la secuencia de salida.

- Position-wise Feed-Forward Networks:

    - Lo que hace es transformar cada vector de representación de manera individual y de manera idéntica en todas las posiciones.


In [1]:
import numpy as np
import tensorflow as tf

def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model) // 2)) / np.float32(d_model))
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

Responda: ¿Qué es el posicional encoding y por qué debe definirse una función para los transformer?

*   Este es importnnte porque hace que el modelo distinga entre diferentes posiciones en la secuencoa en la que se trabaja. Por lo tanto se debe definir ya que calcula un vector asignando un valor único a cada posición de la secuencia.

In [2]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    
    return output, attention_weights

In [3]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        
        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        
        output = self.dense(concat_attention)
        
        return output, attention_weights

In [4]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

In [5]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2

In [6]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()
        
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)
        
        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)
        
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        
        return out3

In [7]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 maximum_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()
        
        self.d_model = d_model
        self.num_layers = num_layers
        
        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, dropout_rate) 
                           for _ in range(num_layers)]
        
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]
        
        # Embedding + Positional Encoding
        x = self.embedding(x)  
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        
        x = self.dropout(x, training=training)
        
        # Pasar por las capas del codificador
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        
        return x  # (batch_size, input_seq_len, d_model)

In [8]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
                 maximum_position_encoding, dropout_rate=0.1):
        super(Decoder, self).__init__()
        
        self.d_model = d_model
        self.num_layers = num_layers
        
        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, dropout_rate) 
                           for _ in range(num_layers)]
        
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}
        
        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        
        x = self.dropout(x, training=training)
        
        # Pasar por las capas del decodificador
        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)
            
            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2
        
        return x, attention_weights  # (batch_size, target_seq_len, d_model), attention weights


In [9]:
# Transformer
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, dropout_rate=0.1):
        super(Transformer, self).__init__()
        
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                               input_vocab_size, pe_input, dropout_rate)
        
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                               target_vocab_size, pe_target, dropout_rate)
        
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask,
             look_ahead_mask, dec_padding_mask):
        # Codificador
        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
        
        # Decodificador
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)  # (batch_size, tar_seq_len, d_model)
        
        # Output final
        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        
        return final_output, attention_weights

# Máscaras (Simplificadas para este ejemplo)
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(seq_len):
    mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    return mask  # (seq_len, seq_len)


In [10]:
#Datos de prueba
num_layers = 4
d_model = 128
num_heads = 8
dff = 512
input_vocab_size = 8500
target_vocab_size = 8000
pe_input = 1000
pe_target = 1000
dropout_rate = 0.1

#Su output shape debe ser: (64, 50, 8000)

In [11]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=input_vocab_size,
    target_vocab_size=target_vocab_size,
    pe_input=pe_input,
    pe_target=pe_target,
    dropout_rate=dropout_rate
)

In [12]:
# Datos de prueba
batch_size = 64
inp_seq_len = 50  # Longitud de la secuencia de entrada
tar_seq_len = 50  # Longitud de la secuencia de salida

# Generar datos de entrada y salida aleatorios
sample_input = tf.random.uniform((batch_size, inp_seq_len), minval=1, maxval=input_vocab_size, dtype=tf.int32)
sample_target = tf.random.uniform((batch_size, tar_seq_len), minval=1, maxval=target_vocab_size, dtype=tf.int32)

# Crear máscaras
enc_padding_mask = create_padding_mask(sample_input)
dec_padding_mask = create_padding_mask(sample_target)  # Dec padding mask should be based on target
look_ahead_mask = create_look_ahead_mask(tar_seq_len)

# Forward Pass
final_output, _ = transformer(
    sample_input, 
    sample_target, 
    training=False, 
    enc_padding_mask=enc_padding_mask, 
    look_ahead_mask=look_ahead_mask, 
    dec_padding_mask=dec_padding_mask
)

print("Shape del Output Final:", final_output.shape)


ValueError: Exception encountered when calling Transformer.call().

[1mOnly input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: False (of type <class 'bool'>)[0m

Arguments received by Transformer.call():
  • inp=tf.Tensor(shape=(64, 50), dtype=int32)
  • tar=tf.Tensor(shape=(64, 50), dtype=int32)
  • training=False
  • enc_padding_mask=tf.Tensor(shape=(64, 1, 1, 50), dtype=float32)
  • look_ahead_mask=tf.Tensor(shape=(1, 1, 50, 50), dtype=float32)
  • dec_padding_mask=tf.Tensor(shape=(64, 1, 1, 50), dtype=float32)